In [2]:
import math
from math import prod
import numpy as np


Problem 1

1.1

In [3]:
class figure:
    def __init__(self, x0, y0):
        self.x_axis = x0
        self.y_axis = y0

    def center_distance(self, x, y):
        self.point_x_axis = x
        self.point_y_axis = y
        return math.sqrt((self.point_x_axis - self.x_axis)**2 + (self.point_y_axis-self.y_axis)**2)


In [4]:
fig = figure(1,0)
print(fig.x_axis, fig.y_axis)

1 0


1.2

In [5]:
class rectangle(figure):
    def __init__ (self, x0, y0, len, wid):
        super(rectangle, self).__init__(x0, y0)
        self.length = len
        self.width = wid

    def area(self):
        return self.length*self.width
    
    def perimeter(self):
        return self.width*2 + self.length *2

In [6]:
rect = rectangle(2,3,3,5)
print ("The area is", rect.area())
print ("The perimeter is", rect.perimeter())
print("The distance from center of the figure to the point is", rect.center_distance(10,10))


The area is 15
The perimeter is 16
The distance from center of the figure to the point is 10.63014581273465


In [7]:
class circle(figure):
    def __init__ (self, x0, y0, rad):
        super(circle, self).__init__(x0, y0)
        self.radius = rad

    def area(self):
        return self.radius**2 * math.pi
    
    def perimeter(self):
        return math.pi * 2*self.radius

In [8]:
circ = circle(5,5,4)
print ("The area is", circ.area())
print ("The perimeter is", circ.perimeter())
print("The distance from center of the figure to the point is", circ.center_distance(10,10))

The area is 50.26548245743669
The perimeter is 25.132741228718345
The distance from center of the figure to the point is 7.0710678118654755


Problem 2.

2.1

In [9]:
class forward:
    def __init__(self,strike,maturity_in_months):
        self.strike = strike
        self.maturity = maturity_in_months*30

    def payoff(self, s0, u, d, r):
        self.s0 = s0
        self.u = u
        self.d = d
        self.r = r
        x = [np.random.choice([1+u, 1+d], p = [(self.r/360 - self.d)/(self.u-self.d), 1-(self.r/360 - self.d)/(self.u-self.d)]) for _ in range (self.maturity)]
        st = s0 * prod(x)
        return st - self.strike

In [10]:
fwd = forward(110, 3)
print ("The forward payoff would be", fwd.payoff(100, 0.01, -0.006, 0.07))

The forward payoff would be -11.45621358000784


2.2


In [11]:
class pricer(forward):
    def __init__(self, strike, maturity_in_months):
        super(pricer, self).__init__(strike, maturity_in_months)
    

In [22]:
class analytic_pricer(pricer):
    def __init__(self, strike, maturity_in_months, s0, r):
        super(analytic_pricer, self).__init__(strike, maturity_in_months)
        self.s0 = s0
        self.r = r
    
    def price(self):
        return self.s0 - self.strike*math.exp(-self.r*self.maturity/360)

In [23]:
an_pricer = analytic_pricer(110, 3, 100, 0.07)
print("The forward analytic price is", an_pricer.price())

The forward analytic price is -8.091745923158058


In [24]:
class mc_pricer(pricer):
    def __init__(self, strike, maturity_in_months, n_paths):
        super(mc_pricer, self).__init__(strike, maturity_in_months)
        self.n_paths = n_paths

    def price(self, s0, u, d, r):
        self.s0 = s0
        self.u = u
        self.d = d
        self.r = r
        
        st = []
        for i in range(self.n_paths):
            np.random.seed(i)
            x = [np.random.choice([1+u, 1+d], p = [(self.r/360 - self.d)/(self.u-self.d), 1-(self.r/360 - self.d)/(self.u-self.d)]) for _ in range (self.maturity)]
            st.append(s0*prod(x))
            payoffs = [s - self.strike for s in st]
            discounted_payoffs = [math.exp(-self.r*self.maturity/360)*payoff for payoff in payoffs]
        return np.mean(discounted_payoffs)


In [25]:
m_pricer = mc_pricer(110, 3, 1000)
print("Monte Carlo method price is", m_pricer.price(100, 0.01, -0.006, 0.07))




Monte Carlo method price is -7.975711505983198


$$
E(\frac{S_T - K}{e^{rT}}) = \frac{1}{e^{rT}}E(S_0*X_1*...*X_T) - \frac{K}{e^{rT}} = \frac{S_0}{e^{rT}}E(X_1)*...*E(X_T) - \frac{K}{e^{rT}} = \frac{S_0}{e^{rT}} \prod^T_{i=1}{E(X_i)} - \frac{K}{e^{rT}}=\\
= \frac{S_0}{e^{rT}} \prod^T_{i=1} {\frac{(1+u)(r-d)}{u-d} + (1+d)(1-\frac{r - d}{u-d})} - \frac{K}{e^{rT}} = \frac{S_0}{e^{rT}} \prod^T_{i=1} {\frac{(1+u)(r-d)}{u-d} + \frac{(1+d)(u-r)}{u-d}} - \frac{K}{e^{rT}} = \frac{S_0}{e^{rT}} \prod^T_{i=1} {\frac{r+ur-d-ud + u+ud-r-rd}{u-d}} - \frac{K}{e^{rT}}=\\
= \frac{S_0}{e^{rT}} \prod^T_{i=1} {\frac{u(r+1)-d(r+1)}{u-d}} - \frac{K}{e^{rT}} = \frac{S_0}{e^{rT}} \prod^T_{i=1} {r+1} - \frac{K}{e^{rT}} = \frac{S_0}{e^{rT}} (r+1)^T - \frac{K}{e^{rT}} = \frac{S_0}{e^{rT}} e^{rT} - \frac{K}{e^{rT}} = S_0 - \frac{K}{e^{rT}}
$$
Since $(r+1)^T \overset{T \rightarrow \infty}{\longrightarrow} e^{rT}$